# Import Needed Modules

pip install spacy

python -m spacy download en_core_web_sm

Use: TensorFlow:2.13 in Vertex AI Jupiter notebook config

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import spacy

# EDA

In [2]:
# Read the dataset with name "Emotion_classify_Data.csv" and store it in a variable df
df = pd.read_csv("gcs://emotion-classify-bucket/Emotion_classify_Data.csv")

# Print the shape of dataframe
print(df.shape)

# Print top 5 rows
df.head(5)

(5937, 2)


,Comment,Emotion
0,i seriously hate one subject to death but now ...,fear
1,im so full of life i feel appalled,anger
2,i sit here to write i start to dig out my feel...,fear
3,ive been really angry with r and i feel like a...,joy
4,i feel suspicious if there is no one outside l...,fear


In [3]:
# Check the distribution of Emotion
df['Emotion'].value_counts()

Emotion
anger    2000
joy      2000
fear     1937
Name: count, dtype: int64

In [4]:
# Show sample
print(f"{df['Comment'][0]} -> {df['Emotion'][0]}")

i seriously hate one subject to death but now i feel reluctant to drop it -> fear


# Preprocessing

In [5]:
# load english language model and create nlp object from it
nlp = spacy.load("en_core_web_sm") 

In [6]:
txt = df['Comment'][3]
txt

'ive been really angry with r and i feel like an idiot for trusting him in the first place'

In [7]:
# Tokenization
doc = nlp(txt)

### Sentence Tokenization
We won't do this process as data in dataframe is splitted in sentences

In [8]:
# for sentence in doc.sents:
#     print(sentence)

### Word Tokenization

In [9]:
for token in doc:
    print(token)

i
ve
been
really
angry
with
r
and
i
feel
like
an
idiot
for
trusting
him
in
the
first
place


### Stemming and Lemmatization

In [10]:
for token in doc:
    print(f"Word: {token} | -> {token.lemma_}")

Word: i | -> I
Word: ve | -> have
Word: been | -> be
Word: really | -> really
Word: angry | -> angry
Word: with | -> with
Word: r | -> r
Word: and | -> and
Word: i | -> I
Word: feel | -> feel
Word: like | -> like
Word: an | -> an
Word: idiot | -> idiot
Word: for | -> for
Word: trusting | -> trust
Word: him | -> he
Word: in | -> in
Word: the | -> the
Word: first | -> first
Word: place | -> place


### Stop Words

In [11]:
for token in doc:
    if token.is_stop or token.is_punct:
        print(token)

i
been
really
with
and
i
an
for
him
in
the
first


### **Preprocess Function**

In [12]:
# use this utility function to get the preprocessed text data
def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens) 

In [13]:
print(txt)
procces_txt = preprocess(txt)
print(procces_txt)

ive been really angry with r and i feel like an idiot for trusting him in the first place
have angry r feel like idiot trust place


## Apply preprocess function on dataframe

In [14]:
df['preprocessed_comment'] = df['Comment'].apply(preprocess) 

In [15]:
df

,Comment,Emotion,preprocessed_comment
0,i seriously hate one subject to death but now ...,fear,seriously hate subject death feel reluctant drop
1,im so full of life i feel appalled,anger,m life feel appalled
2,i sit here to write i start to dig out my feel...,fear,sit write start dig feeling think afraid accep...
3,ive been really angry with r and i feel like a...,joy,have angry r feel like idiot trust place
4,i feel suspicious if there is no one outside l...,fear,feel suspicious outside like rapture happen
...,...,...,...
5932,i begun to feel distressed for you,fear,begin feel distressed
5933,i left feeling annoyed and angry thinking that...,anger,leave feel annoyed angry thinking center stupi...
5934,i were to ever get married i d have everything...,joy,marry d ready offer ve get club perfect good l...
5935,i feel reluctant in applying there because i w...,fear,feel reluctant apply want able find company kn...


## Encoding target column

In [16]:
df['Emotion_num'] = df['Emotion'].map({'joy' : 0, 'fear': 1, 'anger': 2})

df.head(5)

,Comment,Emotion,preprocessed_comment,Emotion_num
0,i seriously hate one subject to death but now ...,fear,seriously hate subject death feel reluctant drop,1
1,im so full of life i feel appalled,anger,m life feel appalled,2
2,i sit here to write i start to dig out my feel...,fear,sit write start dig feeling think afraid accep...,1
3,ive been really angry with r and i feel like a...,joy,have angry r feel like idiot trust place,0
4,i feel suspicious if there is no one outside l...,fear,feel suspicious outside like rapture happen,1


## Split data into train and test

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df['preprocessed_comment'], df['Emotion_num'], 
                                                    test_size=0.2, random_state=42, stratify=df['Emotion_num'])

In [18]:
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)

Shape of X_train:  (4749,)
Shape of X_test:  (1188,)


## Convert text column to numeric vector

In [19]:
v = TfidfVectorizer()

X_train_cv = v.fit_transform(X_train)
X_test_cv = v.transform(X_test)

# All TfidfVectorizer vocabularies
print(v.vocabulary_)

{'thrilled': 5451, 'lot': 3194, 'thing': 5439, 'feel': 1976, 'petrified': 3942, 'know': 3004, 'good': 2297, 'look': 3182, 'honor': 2548, 'word': 6040, 'help': 2489, 'start': 5112, 'way': 5927, 'trust': 5601, 'come': 982, 'quarter': 4250, 'invigorated': 2831, 'work': 6041, 'end': 1724, 'spring': 5077, 'sleep': 4890, 'eat': 1638, 'take': 5328, 'care': 758, 'firm': 2026, 'believer': 491, 'make': 3246, 'woman': 6031, 'terrific': 5405, 'great': 2336, 'trip': 5587, 'salon': 4615, 'lift': 3118, 'spirit': 5053, 'bit': 537, 'attempt': 353, 'hair': 2392, 'color': 971, 'session': 4746, 'later': 3043, 'brave': 634, 'crazy': 1176, 'saturday': 4637, 'night': 3624, 'ask': 317, 'shed': 4776, 'like': 3123, 'say': 4643, 'nt': 3661, 'mind': 3394, 'suggestion': 5239, 'have': 2448, 'try': 5606, 'day': 1282, 'entertained': 1758, 'comfortable': 986, 'think': 5440, 'fit': 2028, 'love': 3203, 'talk': 5336, 'patient': 3878, 'nervous': 3594, 'minute': 3403, 'confuse': 1054, 'wait': 5883, 'long': 3179, 'time': 54

# Machine Learning Model

## 1. Naive Bayes

In [20]:
NB_model = MultinomialNB()

# Model training
NB_model.fit(X_train_cv, y_train)

MultinomialNB()

In [21]:
# Get prediction
y_pred = NB_model.predict(X_test_cv)

In [22]:
# Print accuracy score
print(accuracy_score(y_test, y_pred))

0.9023569023569024


In [23]:
# Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.89      0.89       400
           1       0.91      0.90      0.91       388
           2       0.90      0.92      0.91       400

    accuracy                           0.90      1188
   macro avg       0.90      0.90      0.90      1188
weighted avg       0.90      0.90      0.90      1188



## 2. Random Forest

In [24]:
RFC_model = RandomForestClassifier()

RFC_model.fit(X_train_cv, y_train)

RandomForestClassifier()

In [25]:
# Get the predictions for X_test and store it in y_pred
y_pred = RFC_model.predict(X_test_cv)

In [26]:
# Print Accuracy
print(accuracy_score(y_test, y_pred))

0.9250841750841751


In [27]:
# Print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93       400
           1       0.93      0.93      0.93       388
           2       0.93      0.90      0.92       400

    accuracy                           0.93      1188
   macro avg       0.93      0.93      0.92      1188
weighted avg       0.93      0.93      0.92      1188



# Test Model

## Get text

In [39]:
test_text = df['Comment'][2028]
test_text

'i feel that the students in this classroom are very hostile towards any display of intellect just like the rest of society'

## Apply preprocess

In [40]:
test_text_processed = [preprocess(test_text)]
test_text_processed

['feel student classroom hostile display intellect like rest society']

## Convert to vector

In [41]:
test_text_vc = v.transform(test_text_processed)

## Get Prediction

In [42]:
test_text = RFC_model.predict(test_text_vc)

## Output

In [43]:
print(f"{df['Emotion'][2007]} -> {df['Emotion_num'][2007]}")

emotion = test_text[0]
if emotion == 0:
    print("Predicted emotion is joy")
elif emotion == 1:
    print("Predicted emotion is fear")
elif emotion == 2:
    print("Predicted emotion is anger")
else:
    print("Undefined emotion")

anger -> 2
Predicted emotion is anger


In [33]:
# anger    2000 - 2
# joy      2000 - 0
# fear     1937 - 1